![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [51]:
def get_latest_trade_analytics_key(object_store):
    keys = object_store.Keys

    # keep only trade analytics files
    analytics_keys = [
        k for k in keys if k.endswith("_trade_analytics.json")
    ]

    if not analytics_keys:
        return None

    # ISO timestamp makes lexicographic sort correct
    return sorted(analytics_keys)[-1]
    
def expectancy(x):
    win_rate = (x > 0).mean()
    avg_win = x[x > 0].mean()
    avg_loss = x[x <= 0].mean()
    return win_rate * avg_win + (1 - win_rate) * avg_loss

In [42]:
import json
import pandas as pd
qb = QuantBook()
# latest_key = get_latest_trade_analytics_key(qb.object_store)
# print(latest_key)
key = 'f0951ccd72cd2566c53b077c8cb50f6b_2024-10-02_00-00-00_trade_analytics.json'

df = None
if qb.object_store.contains_key(key):
    string_data = qb.object_store.read(f"{key}")
    json_data = json.loads(string_data)    
    df = pd.json_normalize(json_data)   # flattens nested dicts into columns

df.head()
print(df.columns.tolist())


In [43]:





df["pnl"] = df["pos.pnl"].astype(float)
df["abs_pnl"] = df["pnl"].abs()
df["win"] = df["pnl"] > 0
df["loss"] = df["pnl"] <= 0
# df["pnl_pct"] = df["pos.pnl_pct"].round(x, 2)

df["is_win"] = df["pos.pnl"] > 0
# df["entry_time"] = pd.to_datetime(df["timing.entry_time"])
# df["exit_time"]  = pd.to_datetime(df["timing.exit_time"])
# df["hold_minutes"] = (df["exit_time"] - df["entry_time"]).dt.total_seconds() / 60

In [44]:
# num_positions = len(df)
# or
num_positions = df.shape[0]
num_wins = df["is_win"].sum()
num_losses = (~df["is_win"]).sum()
win_rate = num_wins / num_positions
win_rate
print(num_positions)
print(num_wins)
print(num_losses)
print(win_rate)

In [45]:

# df["is_win"].value_counts()

num_0_pnl = (df["pos.pnl"] == 0).sum()
print(num_0_pnl)
df["is_win"].value_counts(normalize=True)


In [46]:
avg_win  = df.loc[df["is_win"], "pos.pnl"].mean()
avg_loss = df.loc[~df["is_win"], "pos.pnl"].mean()

expectancy = win_rate * avg_win + (1 - win_rate) * avg_loss
print(avg_win)
print(avg_loss)
print(expectancy)

In [47]:
total_pnl = 100 * df["pos.pnl"].sum()
# fees: -$2,688.00
# net profit: $-3,823.00
# pos.pnl sum: 11.35
print(total_pnl)
print(-336.00 - (-541.00))

In [ ]:

# note entry should be regime filter not selection

df["pos.entry"] = pd.to_datetime(df["pos.position.entry_time"], errors="coerce")
df["pos.exit"] = pd.to_datetime(df["pos.position.exit_time"], errors="coerce")

df["entry_hour"] = df["pos.entry"].dt.hour
df["entry_minute"] = df["pos.entry"].dt.minute
df["entry_minutes_since_open"] = (
    df["entry_hour"] * 60 + df["entry_minute"]
) - (9 * 60 + 30)
df["exit_hour"] = df["pos.exit"].dt.hour
df["exit_minute"] = df["pos.exit"].dt.minute

df["hold_minutes"] = (
    df["pos.exit"] - df["pos.entry"]
).dt.total_seconds() / 60

cols_to_view = [
    "pos.entry",
    "entry_hour",
    "entry_minute",
    "entry_minutes_since_open",
    "pos.exit",
    "hold_minutes"
]

df[cols_to_view].head(10)

In [ ]:
# Don’t include raw datetimes
# Include entry time engineered features
# Exit time is for analysis, not prediction
# ntry time is often a top-3 predictor for 0DTE ICs

# pd.pivot_table(
#     df,
#     values="pnl",
#     index=pd.cut(df["entry_minutes_since_open"], [0,30,90,180,300]),
#     aggfunc=["count", "mean"]
# )

time_bins = [0, 30, 90, 180, 300]

pd.pivot_table(
    df,
    values="pnl",
    index=pd.cut(df["entry_minutes_since_open"], time_bins),
    aggfunc=["count", "mean", expectancy]
)
# since mean and expectancy are equal this means
# Tail losses are not disproportionately large in that time window
# means that time isn't destroying edge

# this shows more clearly:
df.groupby(pd.cut(df["entry_minutes_since_open"], time_bins))["pnl"].agg([
    "count",
    "mean",
    "std",
    "min",
    "max",
    expectancy
])

# results are showing
# Time alone doesn’t kill the strategy — but early entries amplify tail risk.

In [ ]:
# combine time with regime filter is where you will likely see expectancy < edge
pd.pivot_table(
    df,
    values="pnl",
    index=pd.cut(df["entry_minutes_since_open"], time_bins),
    columns=pd.cut(df["pos.position.technicals.current_adx"], [0,15,25,40]),
    aggfunc=["mean",expectancy, "count"]
)

# note
# don't overweight mean vs expectancy - its mostly just a math artifact - exp is weighted avg of wins/losses
# for risk insight you want q01, q05 (tail severity), std, maybe win_rate, possibly max_draw_down_per_bucket (if you can sequence trades)

# early entries
# That’s very bad for an iron condor.
# mean / expectancy = -0.096
# count = 10
# Interpretation:
# Early entry
# Low ADX (false sense of range)
# Market is discovering direction
# This is a kill-switch regime

# (30–90 min) + ADX (15–25): Also bad
# low count though
# mid morning, trend emerging, 

# edge is here:
# direction set, time decays, vol compresses
# ADX 0–15   → +0.029 (count 11)
# ADX 15–25  → +0.041 (count 46)
# ADX 25–40  → +0.042 (count 34)

# midday not enough samples but not great so far
# ADX 15–25 → +0.070 (count 3)
# ADX 25–40 → +0.021 (count 15)

# in these buckets mean = expec
# risk controls are working in these regimes

# DO NOT TRADE if:
# - Entry < 30 minutes AND ADX < 15
# - Entry < 90 minutes AND ADX between 15–25

# base line strat
# AUTO-ALLOW if:
# - Entry ≥ 180 minutes (almost regardless of ADX)
# or allow with conditions
# ALLOW WITH EXTRA FILTERS:
# - Entry 90–180
# - ADX ≥ 15
# - Prefer price near VWAP / low IV expansion


In [ ]:

filtered = df[
    ~(
        ((df.entry_minutes_since_open <= 30) &
         (df["pos.position.technicals.current_adx"] <= 15)) |
        ((df.entry_minutes_since_open <= 90) &
         (df["pos.position.technicals.current_adx"].between(15,25)))
    )
]

filtered["pnl"].agg(["count", "mean", expectancy])

# Why this is a big win
# You removed bad regimes
# Trade count only dropped modestly
# Expectancy stayed positive and stable
# Distribution is now tighter (as we saw earlier)
# This confirms:
# Your edge is regime-based, not random
# The exclusions are doing real work

In [69]:
def worst_5pct(x):
    return x.quantile(0.05)

filtered["pnl"].agg([
    "count",
    "mean",
    expectancy,
    worst_5pct,
    "min"
])

In [70]:
filtered.groupby(
    pd.cut(filtered["entry_minutes_since_open"], [0,30,90,180,300])
)["pnl"].agg([
    "count",
    expectancy,
    worst_5pct,
    "min"
])

In [ ]:
# How to read it
# tail_ratio < 3 → very healthy
# 3–5 → acceptable
# > 5 → hidden blow-up risk
# This tells you:
# “How many average winning trades does one bad day wipe out?”How to read it
# tail_ratio < 3 → very healthy
# 3–5 → acceptable
# > 5 → hidden blow-up risk
# This tells you:
# “How many average winning trades does one bad day wipe out?”

def tail_ratio(x):
    return abs(x.quantile(0.05)) / expectancy(x)

filtered.groupby(
    pd.cut(filtered["entry_minutes_since_open"], [0,30,90,180,300])
)["pnl"].agg([
    expectancy,
    worst_5pct,
    tail_ratio
])

# This is exactly why tail analysis matters for iron condors.
# Let’s interpret it very literally and then turn it into hard trading rules.
# shows red flags which are first window, yellow flag the second window and other two good

In [ ]:
adx_bins = [15, 20, 25, 30, 40, 100]

def adx_bucket_stats(df_slice: pd.DataFrame, label: str):
    out = df_slice.groupby(
        pd.cut(df_slice[adx_col], adx_bins, include_lowest=True)
    )["pnl"].agg(["count", "mean", expectancy, worst_5pct, "min", "max"])
    out["window"] = label
    return out

stats_late  = adx_bucket_stats(late,  "late_180_300")
stats_mid   = adx_bucket_stats(mid,   "mid_90_180")
stats_early = adx_bucket_stats(early, "early_30_90")
stats_open  = adx_bucket_stats(open_, "open_0_30")

pd.concat([stats_open, stats_early, stats_mid, stats_late]).reset_index()

# helps to figure out which time buckets and adx buckets produce best and worse results

In [79]:
filtered[adx_col].isna().mean()
(filtered[adx_col] < 15).mean()
(filtered[adx_col] > 100).mean()

In [ ]:


df["price_vs_vwap_atr"] = (
    df["pos.position.technicals.current_price"]
    - df["pos.position.technicals.current_vwap"]
) / df["pos.position.technicals.current_atr"]

df["bb_position"] = (
    df["pos.position.technicals.current_price"]
    - df["pos.position.technicals.current_bb.lower"]
) / (
    df["pos.position.technicals.current_bb.upper"]
    - df["pos.position.technicals.current_bb.lower"]
)

df["gap_pct"] = (
    df["pos.position.technicals.current_open"]
    - df["pos.position.technicals.prev_day_close"]
) / df["pos.position.technicals.prev_day_close"]


In [54]:
y.value_counts(normalize=True)

In [ ]:
adx_bins = [0, 15, 25, 40, 100]

df.groupby(pd.cut(
    df["pos.position.technicals.current_adx"],
    adx_bins
))["pnl"].agg(["count", "mean", expectancy])

# ADX < 15 → range-bound → condor-friendly
# ADX > 25 → trend risk → expect pain

In [27]:
pd.pivot_table(
    df,
    values="pnl",
    index=pd.cut(df["pos.position.technicals.current_adx"], [0,15,25,40]),
    columns=pd.cut(df["gap_pct"], [-1,-0.01,0.01,1]),
    aggfunc="mean"
)

In [ ]:
# pd.pivot_table(
#     df,
#     values="pnl",
#     index=pd.cut(df["pos.position.technicals.current_adx"], [0,15,25,40]),
#     columns=pd.cut(df["price_vs_vwap_atr"], [-5,-1,1,5]),
#     aggfunc="mean"
# )

# What to look for
# Cells with:
# Count > meaningful threshold (e.g. 30–50)
# Expectancy clearly positive
# Cells that are consistently negative → filter them out

pd.pivot_table(
    df,
    values="pnl",
    index=pd.cut(df["pos.position.technicals.current_adx"], [0,15,25,40]),
    columns=pd.cut(df["price_vs_vwap_atr"], [-5,-1,1,5]),
    aggfunc=[expectancy, "count"]
)

In [ ]:
# what makes money, where do my losses come from?


df.sort_values("pnl").head(50)[
    ["entry_minutes_since_open",
     "pos.position.technicals.current_adx",
     "price_vs_vwap_atr",
     "pnl"]
]

In [ ]:
# Often you’ll discover:
# One regime causes most tail losses
# Removing it boosts expectancy dramatically

losers = df[df["pnl"] < 0]

pd.pivot_table(
    losers,
    values="pnl",
    index=pd.cut(losers["pos.position.technicals.current_adx"], [0,15,25,40]),
    aggfunc=["count", "mean"]
)

In [ ]:
# Simple Rule Simulation (No ML Yet)
# Once you identify 2–3 strong filters:


filtered = df[
    (df["entry_minutes_since_open"].between(30, 180)) &
    (df["pos.position.technicals.current_adx"] < 20) &
    (df["price_vs_vwap_atr"].between(-1, 1))
]

filtered["pnl"].agg(["count", "mean", expectancy])

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
feature_cols = [
    "ic.rr", "ic.defined_risk", "ic.em", "ic.em_ok", "ic.cushion", "ic.cushion_score", "ic.rr_score", "ic.center_score",
    "ic.delta_balance_score", "ic.overall_score", 
    "pos.position.technicals.current_adx", "pos.position.technicals.current_rsi", "pos.position.technicals.current_atr",
    "pos.position.technicals.current_vix", "pos.position.technicals.current_vix1d",
]

feature_cols += [
    "price_vs_vwap_atr",
    "bb_position",
    "gap_pct"
]
# regime_filters = [
#     "pos.position.technicals.current_adx",
#     "pos.position.technicals.current_vix", "pos.position.technicals.current_vix1d",
# ]
X = df[feature_cols].fillna(0)
y = df["is_win"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

clf = DecisionTreeClassifier(max_depth=4, min_samples_leaf=20, random_state=42)
clf.fit(X_train, y_train)

print("train acc:", clf.score(X_train, y_train))
print("test acc:", clf.score(X_test, y_test))

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))